In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
data=pd.read_csv("csv_result-KDDTrain+.csv")

data = data.drop(columns=['id'])

data.columns = data.columns.str.replace("'", "").str.strip()

data['class'] = data['class'].map({'normal': 0, 'anomaly': 1})

# One-hot encode the categorical columns
categorical_columns = ['protocol_type', 'service', 'flag']
data = pd.get_dummies(data, columns=categorical_columns)

data.head()



,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
1,0,146,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,False
3,0,232,8153,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
4,0,199,420,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False


In [2]:
X=data.drop("class",axis=1).values
y=data["class"].values
test_size=0.3
indices=np.arange(len(X))
np.random.shuffle(indices)
x_shuffle=X[indices]
y_shuffle=y[indices]
split=int(0.8*len(X))
X_train=x_shuffle[:split]
X_test=x_shuffle[split:]
y_train=y_shuffle[:split]
y_test=y_shuffle[split:]
type(X_train)

numpy.ndarray

In [3]:
if isinstance(X_train, np.ndarray):
    X_train = torch.from_numpy(X_train.astype(np.float32))

if isinstance(X_test, np.ndarray):
    X_test = torch.from_numpy(X_test.astype(np.float32))

if isinstance(y_train, np.ndarray):
    y_train = torch.from_numpy(y_train.astype(np.float32))

if isinstance(y_test, np.ndarray):
    y_test = torch.from_numpy(y_test.astype(np.float32))
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)
X_train.shape

torch.Size([100778, 122])

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, nFeatures) -> None:
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(nFeatures,1)
    def forward(self,x):
        y_pred=torch.sigmoid(self.linear(x))
        return y_pred
model=LogisticRegression(X_train.shape[1])    
loss=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)
for i in range(1000):
    y_pred=model(X_train)
    # print(y_pred.shape)
    loss(y_pred,y_train).backward()
    # l.backward()
    optimizer.step()
    optimizer.zero_grad()


with torch.no_grad():
    y_pred=model(X_test)
    y_pred_class=y_pred.round()
    acc=y_pred_class.eq(y_test).sum()/float(y_test.shape[0])
    print(f"Accuracy is :{acc.item()*100:4f}")

Accuracy is :91.081566
